In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from drcomp.autoencoder import FullyConnectedAE, MnistConvAE
from drcomp.reducers import AutoEncoder
from drcomp import estimate_intrinsic_dimension
import wandb
import torch
import pickle
from torchvision import datasets, transforms
from skorch.callbacks import LRScheduler, WandbLogger

In [ ]:
mnist_train = datasets.MNIST(
    root="/storage/data", download=True, transform=transforms.ToTensor()
)
mnist_test = datasets.MNIST(
    root="/storage/data", download=True, transform=transforms.ToTensor(), train=False
)

In [ ]:
X_train = mnist_train.data.numpy().astype("float32")
X_train = X_train.reshape(-1, 1, 28, 28)
X_test = mnist_test.data.numpy().astype("float32")
n_samples = X_train.shape[0]
image_size = (28, 28)

In [ ]:
# intrinsic_dim = estimate_intrinsic_dimension(X_train.reshape(n_samples, -1), K=10) # 15
intrinsic_dim = 16

In [ ]:
lr_schedule = LRScheduler(policy="ReduceLROnPlateau")

config = {
    "epochs": 100,
    "batch_size": 100,
}

wandb_run = wandb.init(project="drcomp", group="MNIST")
wandb = WandbLogger(wandb_run)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoEncoder(
    MnistConvAE,
    batch_size=config["batch_size"],
    max_epochs=config["epochs"],
    device=device,
    callbacks=[lr_schedule, WandbLogger(wandb_run)],
)
model.fit(X_train)
wandb_run.finish()
with open("../models/mnist_conv_ae.pkl", "wb") as f:
    pickle.dump(model, f)

In [ ]:
Y = conv_ae.transform(X_train)
X_hat = conv_ae.inverse_transform(Y)

In [ ]:
# conv_ae.evaluate(X_train.reshape(n_samples, -1), Y=Y.reshape(-1, intrinsic_dim), K=10) takes ~12min
# T = 0.9994

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
original = X_train[0].reshape(image_size)
reconstructed = X_hat[0].reshape(image_size)

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(10, 5))
ax1.imshow(original, cmap="gray")
ax2.imshow(reconstructed, cmap="gray")
plt.suptitle(f"Original vs. Reconstructed by Convolutional AE")
plt.show()

In [ ]:
# TODO: look at the latent space Y with TSNE
from sklearn.manifold import TSNE

Y_embedded = TSNE(n_components=2).fit_transform(Y)  # takes ~3min

In [ ]:
plt.scatter(Y_embedded[:, 0], Y_embedded[:, 1], c=mnist_train.targets)
plt.savefig("../figures/mnist_conv_ae_latent_space.png")
plt.show()